In [120]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import time

import base64
import xmltodict

In [121]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [122]:
documents_by_id = {}
es.indices.create(index='myandex')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'myandex'}

In [123]:
def processFile(i):
    prefix = '../hw_1/byweb_for_course/byweb.'
    suffix = '.xml'
    filename = prefix + str(i) + suffix
    with open(filename, 'rb') as f:
        decoded = f.read().decode('cp1251')
        xmldict = xmltodict.parse(decoded)
        for doc in tqdm(xmldict['romip:dataset']['document']):
            try:
                docID = doc['docID']
                documents_by_id[docID] = {}
                url = base64.b64decode(doc['docURL']).decode('cp1251')
                content = base64.b64decode(doc['content']['#text']).decode('cp1251')
                documents_by_id[docID]['url'] = url
                documents_by_id[docID]['content'] = content
            except Exception as e:
                print(e)

In [124]:
for i in range(1):
    processFile(i)

In [125]:
settings_final = {
    'mappings': {
        'properties': {
            'url': {
                'type': 'text'
            },
            'content': {
                'type': 'text'
            }
        }
    },
    "settings": {
    "analysis": {
      "analyzer": {
        "my_custom_analyzer": {
          "type":      "custom", 
          "tokenizer": "standard",
          "char_filter": [
            "html_strip",
            "yont"
          ],
          "filter": [
            "lowercase",
            "asciifolding",
            "russian_snow",
            "english_snow"
          ]
        }
      },
        'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е',
                        'Ё => Е'
                    ]
                }
            },
    'filter': {
            'stop_words': {
                'type': 'stop',
                'stopwords': [
                ]
            },
            'russian_snow': {
                'type': 'snowball',
                'language': 'russian'
            },
            'english_snow': {
                'type': 'snowball',
                'language': 'english'
            }
     }
    }
  }
}

In [126]:
def recreate_index():
    es.indices.delete(index='myandex')
    es.indices.create(index='myandex', body=settings_final)

In [127]:
recreate_index()

In [128]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='myandex', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [129]:
analyzer = {
    'analyzer': 'my_custom_analyzer'
}

check_analyzer(analyzer, '<meta http-equiv="Content-Type" content="text/html; charset=windows-1251"> bla bla русский countable текст Ёшкин кот')

['bla', 'bla', 'русск', 'countabl', 'текст', 'ешкин', 'кот']

In [130]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [131]:
def es_actions_generator():
    for doc_id, doc in tqdm(documents_by_id.items()):
        yield create_es_action('myandex', doc_id, doc)

In [132]:
start = time.time()
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)
end = time.time()
print(f"Time on index creation: {time.strftime('%H:%M:%S.%l', time.gmtime(end - start))}")
print(f"In seconds: {end - start}")


Time on index creation: 00:00:46.12
In seconds: 46.717292070388794


In [133]:
def search(query, *args):
    pretty_print_result(es.search(index='myandex', body=query, size=20), args)
    # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20

def raw_search(query):
    search_result = es.search(index='myandex', body=query, size=20)['hits']
    return [(hit['_id'], hit['_score']) for hit in search_result['hits']]
    
def pretty_print_result(search_result, fields=[]):
    # fields is a list of fields names which we want to be printed
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits'][:6]:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='myandex', id=doc_id)['_source']

In [134]:
def get_query(query):
    return {
    'query': {
        'bool': {
            'must': {
                'match': {
                    'content': query
                }
            }
        }
    }
    }

q = get_query('<meta http-equiv="Content-Type" content="text/html; charset=windows-1251">')
search(q)
raw_search(q)

Total documents: 10000
Doc 90682, score is 0.96447104
Doc 92550, score is 0.9615358
Doc 90748, score is 0.9613672
Doc 93656, score is 0.9582686
Doc 90844, score is 0.95792323
Doc 91118, score is 0.9578218


[('90682', 0.96447104),
 ('92550', 0.9615358),
 ('90748', 0.9613672),
 ('93656', 0.9582686),
 ('90844', 0.95792323),
 ('91118', 0.9578218),
 ('90516', 0.9544537),
 ('91436', 0.95415574),
 ('91108', 0.9540011),
 ('91309', 0.9535908),
 ('90881', 0.95118463),
 ('91295', 0.95053643),
 ('90728', 0.9503658),
 ('268', 0.9474024),
 ('92543', 0.9469462),
 ('91681', 0.9469462),
 ('34557', 0.9415664),
 ('91056', 0.9385573),
 ('33988', 0.93676704),
 ('34796', 0.9366266)]

In [135]:
def print_index_size(index): 
    print(f"Size of index: {es.indices.stats(index)['_all']['primaries']['store']['size_in_bytes'] / 2 ** 30} GB")

In [136]:
print_index_size('myandex')

Size of index: 0.5000467635691166 GB


In [137]:
def load_queries_and_relevance():
    relevance = defaultdict(dict)
    filename = '../or_relevant-minus_table.xml'
    with open(filename, 'rb') as f:
        xmldict = xmltodict.parse(f.read())
        for task in tqdm(xmldict['taskDocumentMatrix']['task']):
            task_rel = {}
            has_vital = False
            for doc in task['document']:
                if doc['@relevance'] == 'vital':
                    has_vital = True
                task_rel[doc['@id']] = doc['@relevance']
            if has_vital:
                relevance[task['@id']] = task_rel
    filename = '../web2008_adhoc.xml'
    with open(filename, 'rb') as f:
        xmldict = xmltodict.parse(f.read())
        for task in tqdm(xmldict['task-set']['task']):
            if task['@id'] in relevance:
                relevance[task['@id']]['querytext'] = task['querytext']
    return relevance

In [138]:
relevance = load_queries_and_relevance()

In [156]:
def get_number_of_correct_out_of_k(results, task_relevance, k):
    return sum([1 if res[0] in task_relevance and task_relevance[res[0]] == 'vital' else 0 for res in results[:k]])

def measure_performance():    
    Q = len(relevance)
    pq = 0
    rq = 0
    prq = 0
    mapq = 0
    for task in relevance.keys():
        sk = 0
        task_relevance = relevance[task]
        results = raw_search(get_query(task_relevance['querytext']))
        #if len(results) < 20:
        #    print("WARNING LESS 20")
        sk = get_number_of_correct_out_of_k(results, task_relevance, 20)
        pq += sk / 20
        relevant_size = len(['vital' for value in task_relevance.values() if value == 'vital'])
        rq += sk / relevant_size
        prq += get_number_of_correct_out_of_k(results, task_relevance, relevant_size) / relevant_size
        mapk = 0
        for k in range(1, 21):
            mapk += get_number_of_correct_out_of_k(results, task_relevance, k) / k
        mapk /= 20
        mapq += mapk
    print(f"p@20: {pq / Q}")
    print(f"r@20: {rq / Q}")
    print(f"p@R(q): {prq / Q}")
    print(f"map@20(q): {mapq / Q}")

In [157]:
measure_performance()

p@20: 0.0952525252525252
r@20: 0.04990920901604512
p@R(q): 0.04481541392225005
map@20(q): 0.1654789225964641
